## Workflow for Fire Alarm Civilian Counting

### This notebook runs through a machine learning workflow for creating a model to be used for couting the number of people exiting a building during a fire disaster or drill from close camera footage. 

1. Data Collection & Preprocessing - The roboflow website contains numerous datasets. After merging a few properly annotated datasets, I produced a more diverse and complete dataset. Most of the preprocessing was done on the roboflow site to ensure annotations and photo resolutions were of proper quality and dimension.
2. Data Ingestion - The dataset is sourced from Roboflow and is an Image Segmentation for detecting human beings.
3. Hyper-parameter tuning - Setting the pretrained model from ultralytics as well as the other paramters such as number of epochs, image size etc to finetune the models performance.  
4. Data Modelling & Training - Training a suitable model on the train set for periods of time while also changing with hyperparameters for effective finetuning.
5. Model Evaluation & Testing - Testing models on new and unseen data and recording metrics and important observations.
6. Model Deployment - Setting up a simple basic webpage to return feedback of sample videos placed into it with the use of API's to return people statistics and other information that can be drawn from the use case.     


#### 0. Workspace Set-Up

In [5]:
# Install Packages

!pip install ultralytics
!pip install roboflow


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: C:\Users\HP\AppData\Local\Programs\Python\Python311\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: C:\Users\HP\AppData\Local\Programs\Python\Python311\python.exe -m pip install --upgrade pip


In [6]:
# Import Packages

import ultralytics
ultralytics.checks

from IPython.display import display, Image
from ultralytics import YOLO


In [7]:
# GPU utilzation verification (Most likely Google colab or Kaggle GPU)

!nvidia-smi

'nvidia-smi' is not recognized as an internal or external command,
operable program or batch file.


In [8]:
#Point to home directory

import os
HOME = os.getcwd()
print(HOME)

c:\Users\HP\Documents\PAU\Fire_Alarm_Counter


#### 1. Data collection & Data Preprocessing

Link to dataset: https://universe.roboflow.com/terry-chang-ig5jg/people_test/dataset/2

During the collection of data and compilation of the dataset to the roboflow dataset used. For data preprocessing, image augmentations such as rotation and blur has been added to the variety of images in the train set. All images have also been verified for suitable annotations and absence of missing annotations as well.

#### 2. Data Ingestion

In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="hao9lcgXDRIgQVH08weo")
project = rf.workspace("terry-chang-ig5jg").project("people_test")
dataset = project.version(2).download("yolov8")

#### 3. Hyper-parameter Tuning

Number of epochs determines the iteration limits during model training

Patience number to stop iterations when no significatn learning or improvement is detected

Image Size to specify the size of train images being used for scaling purposed

Pretrained model trained on the microsoft COCO model. We will finetune this model using our Image segmentation dataset

In [10]:
parameters = {
    "epochs": 20,
    "patience": 8,
    "image_size": 640,
}

#### 4. Data Modelling & Training 

Initiating the training process with a pretrained model and the location to our dataset along with the hyperparameters we have specified

In [ ]:
!yolo task=segment mode=train model=yolov8n data="Users\HP\Documents\PAU\Fire_Alarm_Counter\TerryChangDataset/data.yaml" epochs = parameters["epochs"] patience = patience["patience"] imgsz = parameters["image_size"]

#### 5. Model Evaluation

Veryfying trained model on validation set for analysis of performance and to obtain metrics

In [ ]:
!yolo task=segment mode=val model={HOME}/runs/segment/train/weights/best.pt data={dataset.location}/data.yaml

##### Testing on video clip 

In [ ]:
import torch
import cv2
import numpy as np

def POINTS(event, x, y, flags, param):
    if event == cv2.EVENT_MOUSEMOVE :  
        colorsBGR = [x, y]
        print(colorsBGR)
        

cv2.namedWindow('ROI')
cv2.setMouseCallback('ROI', POINTS)

cap=cv2.VideoCapture('testvideo.mp4')

model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
count=0
while True:
    ret,frame=cap.read()
    if not ret:
        break
    count += 1
    if count % 3 != 0:
        continue
    
    frame=cv2.resize(frame,(1020,500))
    results = model(frame)
    for index, row in results.pandas().xyxy[0].iterrows():
        x1 = int(row['xmin'])
        y1 = int(row['ymin'])
        x2 = int(row['xmax'])
        y2 = int(row['ymax'])
        d=(row['name'])
        print(d)
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0,0,255), 1)
    cv2.imshow("ROI",frame)
    if cv2.waitKey(1)&0xFF==27:
        break
cap.release()
cv2.destroyAllWindows()


#### 6. Model Deployment